In [56]:
import json
import os
import sys

In [57]:
#load json file
def load_json(file):
    with open(file, 'r') as f:
        return json.load(f)

In [58]:
mattrife_posts_ig = load_json("daisychoii_posts.json")

In [59]:
mattrife_posts_ig.keys()

mattrife_keylist = list(mattrife_posts_ig.keys())

In [60]:
# mattrife_posts_ig[mattrife_keylist[0]] #still includes metadata
POST_NUM = 0
mattrife_posts_ig[mattrife_keylist[POST_NUM]]['items'][0]
# ^ This is the actual post info

{'pk': '3506020729823263076',
 'id': '3506020729823263076_2204228646',
 'fbid': '18366870706113427',
 'device_timestamp': 1732170109552613,
 'caption_is_edited': False,
 'strong_id__': '3506020729823263076_2204228646',
 'deleted_reason': 0,
 'has_shared_to_fb': 0,
 'has_delayed_metadata': False,
 'mezql_token': '',
 'share_count_disabled': False,
 'should_request_ads': False,
 'is_reshare_of_text_post_app_media_in_ig': False,
 'integrity_review_decision': 'pending',
 'client_cache_key': 'MzUwNjAyMDcyOTgyMzI2MzA3Ng==.3',
 'is_visual_reply_commenter_notice_enabled': True,
 'like_and_view_counts_disabled': False,
 'is_post_live_clips_media': False,
 'can_modify_carousel': False,
 'is_quiet_post': False,
 'has_privately_liked': False,
 'filter_type': 0,
 'taken_at': 1732170262,
 'can_see_insights_as_brand': False,
 'media_type': 8,
 'code': 'DCn40cwRXlk',
 'caption': {'bit_flags': 0,
  'created_at': 1732170263,
  'created_at_utc': 1732170263,
  'did_report_as_spam': False,
  'is_ranked_com

In [61]:
post0_json = mattrife_posts_ig[mattrife_keylist[POST_NUM]]['items'][0]

#### Desired data pairs and metadata

In [62]:
#declare a dictionary to store the post info
post0 = {}

In [63]:
post0['id'] = post0_json['id']
post0['shortcode'] = post0_json['code'] #shortcode id
post0['caption'] = post0_json['caption']['text'] #caption

In [64]:
#We're interested in only the first (display) image
post0['head_image_url'] = post0_json['image_versions2']['candidates'][0]['url']

In [65]:
#Also interested in music metadata
music_metadata = post0_json['music_metadata']
post0_json['music_metadata']

{'audio_type': None,
 'music_canonical_id': '0',
 'pinned_media_ids': None,
 'music_info': None,
 'original_sound_info': None}

In [66]:
post0['audio_type'] = music_metadata['audio_type']
music_metadata['audio_type'] == 'licensed_music'
#We want that to be true


False

In [67]:
music_info = music_metadata['music_info']

In [68]:
music_info['music_asset_info']

TypeError: 'NoneType' object is not subscriptable

In [54]:
post0['music_title'] = music_info['music_asset_info']['title']
post0['music_artist'] = music_info['music_asset_info']['display_artist']

TypeError: 'NoneType' object is not subscriptable

#### Parsing and Projecting with JMESPath

In [17]:
! pip install jmespath

In [69]:
import jmespath

def parse_post_json(data: dict) -> dict:
    """Parse json data from instagram post and return a dictionary of relevant information"""
    result = jmespath.search(
        """{
        id: id,
        shortcode: code,
        caption: caption.text,
        head_image_url: image_versions2.candidates[0].url,
        audio_type: music_metadata.audio_type,
        music_title: music_metadata.music_info.music_asset_info.title,
        music_artist: music_metadata.music_info.music_asset_info.display_artist
    }""",
        data,
    )
    return result

In [70]:
parse_post_json(post0_json)

{'id': '3506020729823263076_2204228646',
 'shortcode': 'DCn40cwRXlk',
 'caption': 'cozy paris 🧸',
 'head_image_url': 'https://scontent-lax3-1.cdninstagram.com/v/t51.29350-15/467698469_1994928560969829_1353139507592233308_n.jpg?stp=dst-jpg_e35_p1080x1080_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkNBUk9VU0VMX0lURU0uaW1hZ2VfdXJsZ2VuLjE0NDB4MTgwMC5zZHIuZjI5MzUwLmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=109&_nc_oc=Q6cZ2QHG-eO0FSfqDt10XuUlUxXuKgetGmQPjiccBJjsLWs9eeBuCoDtIxqtaVPUPpfqrZQ&_nc_ohc=COw-c4OaWeUQ7kNvwETr0jQ&_nc_gid=e9bzbpBGOwmP41yRKhFMYw&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MzUwNjAyMDcyMTA5OTAyMDQwMQ%3D%3D.3-ccb7-5&oh=00_AfGaZeikeefLidwfkirqj9xOzaocfa6nYs2FZkaxZp6b3w&oe=67FA8A1D&_nc_sid=fc8dfb',
 'audio_type': None,
 'music_title': None,
 'music_artist': None}

In [71]:
# for all posts, parse them, and save them to a new json file
all_posts = []
for post in mattrife_posts_ig.values():
    all_posts.append(parse_post_json(post['items'][0]))

with open('mattrife_posts_parsed.json', 'w') as f:
    json.dump(all_posts, f, indent=4)


In [23]:
! pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 16.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 14.4 MB/s eta 0:00:00a 0:00:01


In [72]:
# alternatively, for all posts, save into a pandas DF, and then into a csv
import pandas as pd

df = pd.DataFrame(all_posts)
df.to_csv('mattrife_posts_parsed.csv', index=False)

In [73]:
df

,id,shortcode,caption,head_image_url,audio_type,music_title,music_artist
0,3506020729823263076_2204228646,DCn40cwRXlk,cozy paris 🧸,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None
1,3563235172982084848_2204228646,DFzJ4LQxlTw,on campus to yap with friends🫶🏻,https://scontent-lax3-2.cdninstagram.com/v/t51...,licensed_music,Dream,"Suzy, BAEKHYUN"
2,3418980747267312513_2204228646,C9yqOAGxIuB,Hope to see you at KCON!!\n\nKCON LA 2024 K-CO...,https://scontent-lax3-2.cdninstagram.com/v/t51...,None,None,None
3,3459609029067120898_2204228646,DADABUHR6EC,good moments from Shanghai <3,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None
4,3503827644955530200_2204228646,DCgGK3TxJPY,None,https://scontent-lax3-1.cdninstagram.com/v/t51...,licensed_music,Chanel,Frank Ocean
...,...,...,...,...,...,...,...
57,3286384313518434717_2204228646,C2blR8Crqmd,hi everyone,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None
58,2976530309318985298_2204228646,ClOwovGJEZS,typa girl,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None
59,3352213963183859501_2204228646,C6FdNa2ract,a love letter to earth ౨ৎ,https://scontent-lax3-2.cdninstagram.com/v/t51...,None,None,None
60,3202854442621530780_2204228646,Cxy0yVnvPKc,idk who lied to you but it’s still the 28th… h...,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None


In [74]:
df.columns

Index(['id', 'shortcode', 'caption', 'head_image_url', 'audio_type',
       'music_title', 'music_artist'],
      dtype='object')

In [75]:
# for all "None" on audio type, remove them
df = df[df['audio_type'] == 'licensed_music']

In [76]:
# remove id, caption, audio type, 
df = df.drop(columns=['id', 'caption', 'audio_type'])

In [77]:
#can you rename shortcode to id?
df = df.rename(columns={'shortcode': 'ID'})

In [78]:
df

,ID,head_image_url,music_title,music_artist
1,DFzJ4LQxlTw,https://scontent-lax3-2.cdninstagram.com/v/t51...,Dream,"Suzy, BAEKHYUN"
4,DCgGK3TxJPY,https://scontent-lax3-1.cdninstagram.com/v/t51...,Chanel,Frank Ocean
5,DCYuI0YAzB_,https://scontent-lax3-2.cdninstagram.com/v/t51...,What If I Go? (feat. Bonzai),Mura Masa
6,DC2lg1Bxl0l,https://scontent-lax3-1.cdninstagram.com/v/t51...,UP (KARINA Solo),aespa
8,C9jU_gqx4cO,https://scontent-lax3-2.cdninstagram.com/v/t51...,Cookie,NewJeans
9,C9ATcCThz0g,https://scontent-lax3-1.cdninstagram.com/v/t51...,Tongue Tied,GROUPLOVE
10,DAfVk7Ex9MY,https://scontent-lax3-1.cdninstagram.com/v/t51...,Once Upon a Dream,Invadable Harmony
11,DE34artRKWu,https://scontent-lax3-2.cdninstagram.com/v/t51...,pixie dust,Dreamy Kid
12,C8wjHPih5lt,https://scontent-lax3-1.cdninstagram.com/v/t51...,Hashirimizu,Yamafutatsu
13,C9_1WhXx7Ai,https://scontent-lax3-2.cdninstagram.com/v/t51...,No Surprises,Radiohead
